In [1]:
%load_ext autoreload
%autoreload 2
notebook_fixed_dir = False

In [2]:
# this cell can only be called once
import os
if not notebook_fixed_dir:
    os.chdir('..')
    notebook_fixed_dir = True
print(os.getcwd())

/home/svcl-oowl/brandon/research/sil_consistent_at_inference


In [3]:
# imports
import pprint
import pickle
import glob
import random
from pathlib import Path
import math

import torch
from tqdm import tqdm
from PIL import Image
import numpy as np
from pytorch3d.renderer import look_at_view_transform
import matplotlib.pyplot as plt
import pandas as pd
import cv2

from utils import utils
from utils import eval_utils
import deformation.losses as def_losses

In [ ]:
# processing pose quality annotations
import xmltodict
pose_annotation_path = "notebooks/pix3d_chair_occnet_pose_quality_annotations.xml"
# processing xml from cvat
# https://github.com/openvinotoolkit/cvat
with open (pose_annotation_path, 'r') as f:
    f = f.read()
d = dict(xmltodict.parse(f))

pose_annotation_dict = {}
pose_quality_stats = {"num_good":0, "num_medium":0, "num_bad":0}
for entry in d['annotations']['image']:
    anno_dict = dict(entry)
    entry_label = dict(anno_dict['tag'])['@label']
    pose_annotation_dict[anno_dict['@name'].replace('.png', '')] = entry_label
    pose_quality_stats["num_{}".format(entry_label)] += 1
    
pprint.pprint(pose_quality_stats)


In [ ]:
def filter_eval_df_by_quality(original_eval_df, pose_quality_dict, allowed_qualities=["good"]):
    instances_to_consider = [instance for instance in pose_quality_dict if pose_quality_dict[instance] in allowed_qualities]    
    filtered_df = original_eval_df[original_eval_df["instance"].isin(instances_to_consider)]
    return filtered_df

In [4]:
def compare_before_after_refinement(results_df_before, results_df_after):
    
    statistics = {"num_3d_iou_improved":0, "avg_improve_amt": 0, "avg_improve_before_2d_iou":0, "num_3d_iou_worse":0,  "avg_worse_amt":0, "avg_worse_before_2d_iou":0}
    improved_instances = []
    worse_instances = []
    comparision_df = pd.DataFrame()
    for instance in list(results_df_after["instance"]):
        if instance in list(results_df_before["instance"]):
            before_result = results_df_before[results_df_before["instance"]==instance]
            before_3d_iou = float(before_result["3d_iou"])
            #before_2d_iou = float(before_result["2d_iou"])
            after_result = results_df_after[results_df_after["instance"]==instance]
            after_3d_iou = float(after_result["3d_iou"])
            #after_2d_iou = float(after_result["2d_iou"])

            #iou_2d_change = after_2d_iou - before_2d_iou
            iou_3d_change = after_3d_iou - before_3d_iou

            if math.isnan(iou_3d_change):
                statistics["num_3d_iou_worse"] += 1
                continue
            elif iou_3d_change > 0:
                statistics["num_3d_iou_improved"] += 1
                statistics["avg_improve_amt"] += iou_3d_change
                #statistics["avg_improve_before_2d_iou"] += before_2d_iou
                improved_instances.append(instance)
            else:
                statistics["num_3d_iou_worse"] += 1
                statistics["avg_worse_amt"] += iou_3d_change
                #statistics["avg_worse_before_2d_iou"] += before_2d_iou
                worse_instances.append(instance)
            comparision_df = comparision_df.append({"instance": instance, "iou_3d_delta": iou_3d_change}, ignore_index=True)

    statistics["avg_improve_amt"] /= statistics["num_3d_iou_improved"]
    statistics["avg_improve_before_2d_iou"] /= statistics["num_3d_iou_improved"]
    statistics["avg_worse_amt"] /= statistics["num_3d_iou_worse"]
    statistics["avg_worse_before_2d_iou"] /= statistics["num_3d_iou_worse"]
            
    pprint.pprint(statistics)
    #print("example improved instances")
    #print(improved_instances[:100])
    #print("example worse instances")
    #print(worse_instances[:100])
    return comparision_df

In [5]:
def filter_eval_df_warnings(original_df, warnings_to_filter=["NaN nodes", "compute_iou_3d: gt mesh < 200 occupancies", "compute_iou_3d: rec mesh < 200 occupancies"]):
    filter_array = []
    warnings_list = original_df["eval_warnings"].tolist()
    for i in range(len(original_df)):
        warnings = warnings_list[i]
        keep=True
        for warning in warnings:
            if warning in warnings_to_filter:
                keep=False
        filter_array.append(keep)
    return original_df[filter_array]

def filter_and_sort_eval_dfs(df1, df2):
    df1 = filter_eval_df_warnings(df1)
    df2 = filter_eval_df_warnings(df2)
    df1_instances = df1["instance"].tolist()
    df2_instances = df2["instance"].tolist()
    
    df1_filters = [(True if df1_instance in df2_instances else False) for df1_instance in df1_instances]
    df1_filtered = df1[df1_filters].sort_values(by="instance")
    
    df2_filters = [(True if df2_instance in df1_instances else False) for df2_instance in df2_instances]
    df2_filtered = df2[df2_filters].sort_values(by="instance")
    
    return df1_filtered, df2_filtered

def update_2d_iou_input_to_bf(original_df, bf_2d_iou_df):
    if (original_df["instance"].tolist() != bf_2d_iou_df["instance"].tolist() ):
        raise ValueError("Instances don't match")
    original_df["2d_iou_input"] = bf_2d_iou_df["2d_iou_input"]
    return original_df
    

In [6]:
# shapenet
#class_ids = ["02691156", "02828884", "02933112", "02958343", "03001627", "03211117", "03636649", "03691459", "04090263", "04256520", "04379243", "04401088", "04530566"]
class_ids = ["04090263"]
# pix2mesh
evaluation_dir_before_temp = "/home/svcl-oowl/brandon/research/Pixel2Mesh/rec_files/pytorch3d_shapenet_renders/{}/rgba"
evaluation_dir_after_temp = "data/refinements/shapenet_pix2mesh_refinements/{}_pose/{}"
# occnet
#evaluation_dir_before_temp = "/home/svcl-oowl/brandon/research/occupancy_networks/out/pytorch3d_renders/{}/generation/meshes"
#evaluation_dir_after_temp = "data/refinements/shapenet_occnet_refinements/{}_pose/{}"

# pix3d
#evaluation_dir_before_temp = "/home/svcl-oowl/brandon/research/occupancy_networks/out/pix3d/{}/generation/meshes"
#evaluation_dir_after_temp = "data/refinements/pix3d_occnet_refinements/{}_pose/{}"
#class_ids = ["bed", "bookcase", "chair", "desk", "misc", "sofa", "table","tool", "wardrobe"]

pose_type = "gt"

In [7]:
for class_id in class_ids:
    print("------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
    evaluation_dir_before = evaluation_dir_before_temp.format(class_id)
    evaluation_dir_after = evaluation_dir_after_temp.format(pose_type, class_id)
    
    results_df_before_raw = pd.read_pickle(os.path.join(evaluation_dir_before, "eval_results.pkl"))
    results_df_after_raw = pd.read_pickle(os.path.join(evaluation_dir_after, "eval_results.pkl"))
    print(len(results_df_before_raw))
    print(len(results_df_after_raw))
    #if pose_type == "bf":
    #    results_df_before_raw_bf = pd.read_pickle(os.path.join(evaluation_dir_before, "eval_results_bf_pose.pkl"))
    #    results_df_before_raw = update_2d_iou_input_to_bf(results_df_before_raw, results_df_before_raw_bf)

    results_df_before, results_df_after = filter_and_sort_eval_dfs(results_df_before_raw, results_df_after_raw)
    results_df_before["chamfer_L1"] = results_df_before["chamfer_L1"] * 10
    results_df_after["chamfer_L1"] = results_df_after["chamfer_L1"] * 10

    metrics = ["2d_iou_input", "2d_iou_multi", "3d_iou", "chamfer_L1"]
    delta_df = results_df_after[metrics].reset_index(drop=True) / results_df_before[metrics].reset_index(drop=True)
    delta_df = delta_df.replace([np.inf, -np.inf], np.nan)
    delta_df["chamfer_L1"] = delta_df["chamfer_L1"].apply(lambda x: (np.nan if x > 50 else x))
    delta_df["instance"] = results_df_after["instance"].reset_index(drop=True)
    
    print("{} -> {}".format(evaluation_dir_before, evaluation_dir_after))
    print("Note: filtering results from {} to {}\n".format(max(len(results_df_before_raw), len(results_df_after_raw)), len(results_df_after)))
    #print(results_df_before.mean())
    #print("\n -> \n")
    #print(results_df_after.mean())
    print("\n Delta: \n")
    print((delta_df).mean(skipna=True))
    #print((delta_df).median(skipna=True))
    
    print("\n")
    print("most improved: {}".format(delta_df.sort_values("3d_iou", ascending=False)["instance"].to_list()[:20]))
    #with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    #    display(delta_df.sort_values("chamfer_L1", ascending=False))
    
    
    print("\n")
    for metric in ["2d_iou_multi", "3d_iou", "chamfer_L1"]:
        if metric in ["2d_iou_multi", "3d_iou"]:
            num_improved = len(delta_df[delta_df[metric]>1])
            avg_improve_amt = delta_df[delta_df[metric]>1][metric].mean()
            num_worse = len(delta_df[delta_df[metric]<1])
            avg_worse_amt = delta_df[delta_df[metric]<1][metric].mean()
        else:
            num_improved = len(delta_df[delta_df[metric]<1])
            avg_improve_amt = delta_df[delta_df[metric]<1][metric].mean()
            num_worse = len(delta_df[delta_df[metric]>1])
            avg_worse_amt = delta_df[delta_df[metric]>1][metric].mean()
        print("{} -- percent improved: {} avg amount: {}".format(metric, num_improved/(num_improved+num_worse), avg_improve_amt))
        print("{} -- percent worse: {} avg amount: {}".format(metric, num_worse/(num_improved+num_worse), avg_worse_amt))
        

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
474
474
/home/svcl-oowl/brandon/research/Pixel2Mesh/rec_files/pytorch3d_shapenet_renders/04090263/rgba -> data/refinements/shapenet_pix2mesh_refinements/gt_pose/04090263
Note: filtering results from 474 to 416


 Delta: 

2d_iou_input    1.260621
2d_iou_multi    1.297079
3d_iou          2.315801
chamfer_L1      0.482364
dtype: float64


most improved: ['e42d10c193cce254719aed4531d5366b', 'cf21d9de1440d3d203f35d9b3b48203', 'd2aef97be32d8118433d7c8e9cebd7c2', 'fe286de2443835694aa96bdf46776318', 'd9eede71f6941a7c766069834b8696c7', 'd75ba8d56bf868b75a950bb73f1dbae4', 'faaa98e92d722d0ba7dd22c0aa3913f0', 'f7ce17ee88cdec33973c46c285b944d7', 'ccf770542367f03f276c3d558130f8b4', 'cf51da336a5414031b8fee1b14934c0e', 'e9ee67ab2cccdba5771dea817d9f8136', 'faf6a45b04c24fab1d9b004d9d8d2781', 'e574eb37ea6ee685e1f2a1daf1

In [ ]:
display(delta_df)
delta_df.plot.scatter(x="2d_iou_input", y="2d_iou_multi")
delta_df.plot.scatter(x="2d_iou_input", y="3d_iou")
delta_df.plot.scatter(x="2d_iou_input", y="chamfer_L1")

In [ ]:
#display(results_df_before)
iou_2d_delta_threshold = 0.17
filtered_delta_df = delta_df[delta_df["2d_iou_input"]<iou_2d_delta_threshold]
print((filtered_delta_df).mean())
print(len(filtered_delta_df))

In [ ]:
comp_df = compare_before_after_refinement(results_df_before, results_df_after)
comp_df.hist(column="iou_3d_delta", bins=100)